In [1]:
import numpy as np
import pandas as pd
import fuzzymatcher

pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 100


In [2]:
def print_col_names(df):
    for col in df.columns:
        print(col)

In [4]:

def merge_data(opinions_df, aoc_df, judge2_df):
    opinions_cols = [
    #     'docket_num',
        'docket_num_standardized',
    #     'docket_num_normalized',
    #     'full_case_name',
    #     'short_case_name',
    #     'party_1',
    #     'party_2',
        'court_name',
    #     'court_code',
    #     'this_case_cite',
    #     'this_cite_definition',
    #     'this_cite_page_scheme',
        'case_opinion_by',
        'judges',
    #     'decision_date_str',
        'decision_date',
    #     'filing_date_str',
        'filing_date',
    #     'dc_creator',
    #     'dc_identifier',
    #     'dc_identifier_scheme',
    #     'dc_source',
    #     'dc_date',
    #     'pagination_scheme',
    #     'cited_docs',
    #     'doc_details',
    #     'classes'
    ]


    aoc_cols = [
        'CIRCUIT',
        'FILEDATE',
        'JURIS',
        'NOS',
        'ORIGIN',
        'RESIDENC',
        'CLASSACT',
        'DEMANDED',
        'TERMDATE',
        'DISP',
        'PROCPROG',
        'NOJ',
        'AMTREC',
        'JUDGMENT',
        # 'MAGISINV',
        # 'OTHERINV',
        'TAPEYEAR',
        'docketnumber',
        'district',
        'office',
    #     'termjudg',
    #     'filejudg',
    #     'filemag',
        'county',
        # 'casename',
        'fdateuse',
    #     'termmag',
        'tdateuse',
        'TRANSDAT',
        'TRCLACT',
        # 'DJOINED',
        # 'PRETRIAL',
        # 'TRIBEGAN',
        # 'TRIALEND',
        'PROSE',
        # 'titl',
        # 'section',
        # 'subsect',
        'arbit',
        # 'mdldock',
        # 'plt',
        # 'def',
        'transoff',
        # 'transdoc',
        # 'transorg',
        'trmarb',
        'ifp',
        'statuscd'
    ]


    judge_cols = [
        'songername',
        'party',
        'party_Updated',
        'partyaffiliationofrenompres',
        'partyaffiliationofpresident',
        'political_party'
    ]

    judge2_cols = [
        'Judge Name',
        'Court Name',
        'Party of Appointing President'
    ]


    opinions_feature_df = opinions_df[opinions_cols]
    aoc_feature_df = aoc_df[aoc_cols]
    judge_feature_df = judge_df[judge_cols]
    judge2_feature_df = judge2_df[judge2_cols]


    opinions_feature_df['judge_name'] = opinions_feature_df['case_opinion_by'].str.lower().str.replace(' ', '')
    judge2_feature_df['judge_name'] = judge2_feature_df['Judge Name'].str.lower().str.replace(' ', '')
    opinions_feature_df['decision_date'] = opinions_feature_df['decision_date'].astype('datetime64[ns]')


    y = fuzzymatcher.fuzzy_left_join(
        opinions_feature_df,
        judge2_feature_df,
        left_on = 'judge_name',
        right_on = 'judge_name'
    )

    z = y[(y['judge_name_left'].notnull()) & (y['judge_name_right'].notnull())]
    z = z[z['best_match_score'] > 0]

    merged_df = pd.merge(
        z,
        aoc_feature_df,
        left_on = ['docket_num_standardized', 'decision_date'],
        right_on = ['docketnumber', 'TERMDATE']
    )

    merged_df = merged_df[[
    #     'best_match_score',
    #     '__id_left',
    #     '__id_right',
    #     'docket_num_standardized',
    #     'court_name',
    #     'case_opinion_by',
    #     'judges',
        'decision_date',
        'filing_date',
    #     'judge_name_left',
    #     'Judge Name',
        'Court Name',
        'Party of Appointing President',
    #     'judge_name_right',
        'CIRCUIT',
    #     'FILEDATE',
        'JURIS',
        'NOS',
        'ORIGIN',
        'RESIDENC',
        'CLASSACT',
        'DEMANDED',
        'TERMDATE',
        'DISP',
        'PROCPROG',
        'NOJ',
        'AMTREC',
        'JUDGMENT',
        'TAPEYEAR',
    #     'docketnumber',
        'district',
        'office',
        'county',
    #     'fdateuse',
    #     'tdateuse',
    #     'TRANSDAT',
        'TRCLACT',
        'PROSE',
        'arbit',
        'transoff',
        'trmarb',
        'ifp',
        'statuscd',
    ]]
    
        
    outfile = f'/scratch/ayl316/ttml_mr_data/processed_data/{file_name}.csv'
    merged_df.to_csv(outfile, index = False)

In [5]:
import time

opinions_file_names = [
    '6411',
    '6412',
    '6413',
    '6414',
    '6415',
    '6416',
    '6417',
    '6419',
    '6420',
    '6421',
    '6422'
]


aoc_df = pd.read_stata('/scratch/ns4486/motivated-reasoning/data/aoc_district/districtAOC_dupldropped.dta')
judge_df = pd.read_stata('/scratch/ayl316/ttml_mr_data/judges/JudgesBioReshaped_TOUSE.dta')
judge2_df = pd.read_csv('/scratch/ayl316/ttml_mr_data/judges/federal-judicial-service.csv')

for file_name in opinions_file_names:
    print('processing ' + str(file_name))
    start_time = time.time()
    opinions_df = pd.read_json(f'/scratch/ayl316/ttml_mr_data/xml_jsons/{file_name}.json', lines = True)
    merge_data(opinions_df, aoc_df, judge2_df)
    end_time = time.time()
    
    del opinions_df
    
    print('processed ' + str(file_name) + ' in ' + str((end_time - start_time) / 60) + ' mins')

/ext3/miniconda3/lib/python3.8/site-packages/pandas/io/stata.py:1433: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


processing 6414


<ipython-input-4-47f32a0970dc>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions_feature_df['judge_name'] = opinions_feature_df['case_opinion_by'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judge2_feature_df['judge_name'] = judge2_feature_df['Judge Name'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

processed 6414 in 23.391019825140635 mins
processing 6415


<ipython-input-4-47f32a0970dc>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions_feature_df['judge_name'] = opinions_feature_df['case_opinion_by'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judge2_feature_df['judge_name'] = judge2_feature_df['Judge Name'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

...

processed 6420 in 18.631577614943186 mins
processing 6421


<ipython-input-4-47f32a0970dc>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions_feature_df['judge_name'] = opinions_feature_df['case_opinion_by'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judge2_feature_df['judge_name'] = judge2_feature_df['Judge Name'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

processed 6421 in 31.30998051961263 mins
processing 6422


<ipython-input-4-47f32a0970dc>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions_feature_df['judge_name'] = opinions_feature_df['case_opinion_by'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judge2_feature_df['judge_name'] = judge2_feature_df['Judge Name'].str.lower().str.replace(' ', '')
<ipython-input-4-47f32a0970dc>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

processed 6422 in 7.63101087808609 mins


In [7]:
all_df = None

for file_name in opinions_file_names:
    df = pd.read_csv(f'/scratch/ayl316/ttml_mr_data/processed_data/{file_name}.csv')
    
    if all_df is None:
        all_df = df
    else:
        all_df = pd.concat([all_df, df], ignore_index = True)



/ext3/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/ext3/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/ext3/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219459 entries, 0 to 1219458
Data columns (total 28 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   decision_date                  1219459 non-null  object 
 1   filing_date                    907546 non-null   object 
 2   Court Name                     1219459 non-null  object 
 3   Party of Appointing President  1208959 non-null  object 
 4   CIRCUIT                        1219459 non-null  float64
 5   JURIS                          1219459 non-null  float64
 6   NOS                            1219459 non-null  float64
 7   ORIGIN                         1219459 non-null  float64
 8   RESIDENC                       1219459 non-null  float64
 9   CLASSACT                       1219459 non-null  float64
 10  DEMANDED                       1219459 non-null  float64
 11  TERMDATE                       1219459 non-null  object 
 12  DISP          

In [15]:
all_df.to_csv('/scratch/ayl316/ttml_mr_data/processed_data/cases.csv.zip', compression = 'zip', index = False)

In [16]:
df = pd.read_csv('/scratch/ayl316/ttml_mr_data/processed_data/cases.csv.zip')

/ext3/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,20,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
df.shape

(1219459, 28)